In [1]:
 '''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [3]:
NUM_VERTIHUBS = 12 #dependent on trip_data
FLIGHT_SPEED = 60 #m/s

In [4]:
'''
Trip data formatting
'''
trip_data = pd.read_csv('data/Realtime/OpsLimits/trips.csv')
trip_data.drop(trip_data[trip_data.OriginVertihub == trip_data.DestinationVertihub].index, inplace=True) #drop trips where the origin and destination are the same vertiport, reduced from 4801 trips to 1692 with 10 vertiports
trip_data

,Time,OriginVertihub,OriginVertiport,DestinationVertihub,DestinationVertiport
6,0,11,25,5,7
8,0,5,32,11,47
9,0,7,15,5,23
11,0,11,40,1,14
12,0,2,13,11,35
...,...,...,...,...,...
4792,0,5,32,11,47
4793,0,2,0,7,5
4795,0,11,25,4,20
4796,0,4,20,1,34


In [10]:
vertiports_data = pd.read_csv('data/Realtime/OpsLimits/vertiports.csv')
vertiports_data

,vertiport,long,lat,vertihub
0,0,-88.121882,42.418511,2
1,1,-87.773027,41.745196,1
2,2,-88.294468,42.020224,5
3,3,-86.999222,41.413726,10
4,4,-88.408770,41.337693,4
5,5,-88.642812,42.321158,7
6,6,-87.780088,41.876152,11
7,7,-88.066057,41.894364,5
8,8,-87.594271,41.259799,9
9,9,-89.068053,41.836222,0


In [15]:
'''
vertiport statistics
'''
vertiports_per_vertihub = [0 for i in range(NUM_VERTIHUBS)]
for index, vertiport in vertiports_data.iterrows():
    vertiports_per_vertihub[int(vertiport['vertihub'])] += 1
assert(sum(vertiports_per_vertihub) == vertiports_data.shape[0])
vertiports_per_vertihub



[4, 6, 4, 2, 5, 6, 2, 5, 3, 2, 4, 7]

In [6]:
from math import cos, asin, sqrt, pi

# def dist(x1, y1, x2, y2):
#     return math.sqrt((y2-y1)**(y2-y1) + (x2-x1)**(x2-x1))
def ll_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...   
    

In [9]:
'''
trip statistics
'''

arrivals_per_vertihub = [[] for i in range(NUM_VERTIHUBS)]
flight_popularity = dict()
# get the trip arrival time per vertihub
for index, trip in trip_data.iterrows():
    #calculate time of arrival (begin at trip.time), calculated between vertiports
    origin_port = vertiports_data.iloc[trip['OriginVertiport']]
    destination_port = vertiports_data.iloc[trip['DestinationVertiport']]
    if((origin_port['vertiport'],destination_port['vertiport']) not in flight_popularity.keys()):
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] = 1
    else:
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] += 1
    #trip distance in kilometers
    dist = ll_distance(origin_port['lat'], origin_port['long'], destination_port['lat'], destination_port['long'])
    travel_time = (dist*1000)/FLIGHT_SPEED # (km * 1000)/(m/s) -> seconds
    arrival_time = trip['Time'] + travel_time #takeoff time + travel_time = arrival time
    #add arrival time to vertihub
    arrivals_per_vertihub[trip['DestinationVertihub']].append(arrival_time)

# for each list of vertihub, calculate the minimum, average, and maximum distance between arrivals
differences_per_vertihub = []
# simulated_arrivals
for arrivals_list in arrivals_per_vertihub:
    sorted_arrivals = sorted(arrivals_list)
    differences = []
    for index in range(len(sorted_arrivals)-1):
        differences.append(sorted_arrivals[index+1] - sorted_arrivals[index])
    print(sorted_arrivals)
    differences_per_vertihub.append(sorted(differences))

for index, differences in enumerate(differences_per_vertihub):
    print('vertihub ' + str(index) + ' has minimum arrival difference of ' + str(differences[0]))
    print('vertihub ' + str(index) + ' has average arrival difference of ' + str(sum(differences)/len(differences)))
    print('vertihub ' + str(index) + ' has maximum arrival difference of ' + str(differences[len(differences)-1]))
    print('vertihub ' + str(index) + ' has ' + str(len(differences)) + ' trips')
    print('')
   

formatted_flight_popularity = [(flight_popularity[key], key) for key in flight_popularity.keys()]
print('Most popular flight path ' + str((sorted(formatted_flight_popularity))[len(formatted_flight_popularity)-1]))
print('Total flight paths ' + str(len(formatted_flight_popularity)))
avg = sum([num for num, thing in formatted_flight_popularity])/len(formatted_flight_popularity)
print('Mean requests per flight path ' + str(avg))
    
    

[372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 372.40723364254757, 449.69696178189184, 449.69696178189184, 449.69696178189184, 449.69696178189184, 449.69696178189184, 449.69696178189184, 449.69696178189184, 449.69696178189184, 449.69696178189184, 516.6773305708724, 516.6773305708724, 566.0123383194282, 651.0498948394275, 651.0498948394275, 657.5733379273644, 657.5733379273644, 657.5733379273644, 657.5733379273644, 657.5733379273644, 675.0884926582399, 675.0884926582399, 675.0884926582399, 710.7611595372102, 710.7611595372102, 710.7611595372102, 710.7611595372102, 710.7611595372102, 710.7611595372102, 710.7611595372102, 710.7611595372102, 710.7611595372102, 710.7611595372102, 729.6697352206156, 740.89830953019, 740.89830953019, 740.89830953019, 741.5806413068761, 741.5806413068761, 741.5806413068761, 764.12840413783